# 07 – Neural Network Model

In [ ]:
import pickle, numpy as np, pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense

In [ ]:
# load feature vectors
with open('../../data/processed/features_fft_512.pkl','rb') as f:
    features = pickle.load(f)
# reload grouped meta
import json, pickle
with open('../../data/interim/registros.pkl','rb') as f:
    registros = pickle.load(f)
# map base file to meta
meta = {r['Archivo'].rsplit('_',1)[0]:r for r in registros}
X, y_f, y_m, y_v = [], [], [], []
for name, vec in features.items():
    if name in meta:
        X.append(vec)
        m = meta[name]
        y_f.append(m['Falla']); y_m.append(m['Mag']); y_v.append(m['Vs'])
X = np.array(X)

In [ ]:
enc_f = LabelEncoder().fit(y_f); enc_m = LabelEncoder().fit(y_m); enc_v = LabelEncoder().fit(y_v)
Y_f = to_categorical(enc_f.transform(y_f))
Y_m = to_categorical(enc_m.transform(y_m))
Y_v = to_categorical(enc_v.transform(y_v))
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, yf_tr, yf_te, ym_tr, ym_te, yv_tr, yv_te = train_test_split(
    X_scaled,Y_f,Y_m,Y_v,test_size=0.2,random_state=42)

In [ ]:
inp = Input(shape=(X.shape[1],))
x = Dense(256,activation='relu')(inp)
x = Dense(128,activation='relu')(x)
out_f = Dense(Y_f.shape[1],activation='softmax',name='falla')(x)
out_m = Dense(Y_m.shape[1],activation='softmax',name='mag')(x)
out_v = Dense(Y_v.shape[1],activation='softmax',name='vs')(x)
model = Model(inp,[out_f,out_m,out_v])
model.compile(optimizer='adam',
              loss={'falla':'categorical_crossentropy','mag':'categorical_crossentropy','vs':'categorical_crossentropy'},
              metrics={'falla':'accuracy','mag':'accuracy','vs':'accuracy'})


In [ ]:
history = model.fit(X_train,[yf_tr,ym_tr,yv_tr],
                    validation_split=0.2,epochs=20,batch_size=32,verbose=1)

In [ ]:
results = model.evaluate(X_test,[yf_te,ym_te,yv_te],verbose=0)
print('Test accuracy Falla:',results[2]); print('Mag:',results[4]); print('Vs:',results[6])